# Twitter Crawling / Scraping


안되는 듯..

### method 1. Tweepy
using Twitter API

issue: 현재로부터 7일이내의 트윗들만 수집 가능\
과거의 트윗을 수집하려는 해당 프로젝트의 목표와는 맞지 않음!(유료)

### method 2. GetOldTweets3
python library(https://github.com/Mottl/GetOldTweets3)  
과거정보 수집가능


issue: http 404 error\
An error occured during an HTTP request: HTTP Error 404: Not Found\
해당 링크는 유효하지만, getOldTweets3로 access가 안되는 상황


maybe 트위터가 GetOldTweets3가 사용하는 end point를 제거해서..


In [1]:
# GetOldTweet3 사용 준비
try:
    import GetOldTweets3 as got
except:
    !pip install GetOldTweets3
    import GetOldTweets3 as got

In [2]:
# BeautifulSoup4 사용 준비
try:
    from bs4 import BeautifulSoup
except:
    !pip install bs4
    from bs4 import BeautifulSoup

In [20]:
# 가져올 범위를 정의
# 예제 : 2020-09-01 ~ 2020-09-30

import datetime

days_range = []

start = datetime.datetime.strptime("2020-09-01", "%Y-%m-%d")
end = datetime.datetime.strptime("2020-09-30", "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for date in date_generated:
    days_range.append(date.strftime("%Y-%m-%d"))

print("=== 설정된 트윗 수집 기간은 {} 에서 {} 까지 입니다 ===".format(days_range[0], days_range[-1]))
print("=== 총 {}일 간의 데이터 수집 ===".format(len(days_range)))
print("\n=== {}월 한 달간의 데이터 수집 ===".format(int(days_range[0][5:7])))


=== 설정된 트윗 수집 기간은 2020-09-01 에서 2020-09-29 까지 입니다 ===
=== 총 29일 간의 데이터 수집 ===

=== 9월 한 달간의 데이터 수집 ===


In [21]:
# keyword 포함된 트윗 검색
# keyword: 경기

import time

# 수집 기간 맞추기
start_date = days_range[0]
end_date = (datetime.datetime.strptime(days_range[-1], "%Y-%m-%d") 
            + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # setUntil이 끝을 포함하지 않으므로, day + 1

# 트윗 수집 기준 정의
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('경기')\
                                           .setSince(start_date)\
                                           .setUntil(end_date)\
                                           .setMaxTweets(-1)

# 수집 with GetOldTweet3
print("Collecting data start.. from {} to {}".format(days_range[0], days_range[-1]))
start_time = time.time()

tweet = got.manager.TweetManager.getTweets(tweetCriteria)

print("Collecting data end.. {0:0.2f} Minutes".format((time.time() - start_time)/60))
print("=== Total num of tweets is {} ===".format(len(tweet)))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



An error occured during an HTTP request: HTTP Error 404: Not Found
Try to open in browser: https://twitter.com/search?q=%EA%B2%BD%EA%B8%B0%20since%3A2020-09-01%20until%3A2020-09-30&src=typd
Traceback (most recent call last):
  File "/home/hslee/src/getoldtweets3/GetOldTweets3/manager/TweetManager.py", line 355, in getJsonResponse
    response = opener.open(url)
  File "/home/hslee/anaconda3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/home/hslee/anaconda3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/home/hslee/anaconda3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/home/hslee/anaconda3/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/home/hslee/anaconda3/lib/python3.8/urllib/request.py", line 649, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.erro

TypeError: object of type 'NoneType' has no len()

### method 3. Twint

In [22]:
 !python --version

Python 3.8.3


In [23]:
!pip install async-timeout

In [24]:
!pip3 install twint

In [25]:
!pip install nest_asyncio

In [26]:
import nest_asyncio
nest_asyncio.apply()

In [27]:
import twint
word='"경기"' #찾으려는 단어

startDate='2020-09-01' #검색 기준 시작일
endDate='2020-09-02' #검색 기준 종료일
outputFile = 'tweet_'+word+'.csv' #출력할 파일 이름과 형식

def twint_find_new(word, startDate, endDate, outputFile):
  c = twint.Config() # twint config 선언

  # Parameter setting
  c.Limit = 200 # Limit: 총 수집할 트윗의 수를 의미 (모든 트윗을 수집하고 싶다면 옵션을 제거)
  c.Search = word # Limit: 총 수집할 트윗의 수를 의미 (모든 트윗을 수집하고 싶다면 옵션을 제거)
  # (ex. ‘갑분싸 OR “존나 짜증나”‘ 으로 입력할 경우, 갑분싸의 경우에는 이것저것 붙여서 수집되나 '존나 짜증나'는 정확이 따옴표 안 단어를 모두 포함하는 트윗만 수집됨)
  c.Since = startDate
  c.Until = endDate # Since, Until: 수집을 원하는 기간 범위 (끝은 포함하지 않음)
  c.Output = outputFile #Output: 트윗을 수집 후 저장할 때 파일 이름
  c.Popular_tweets = True #Popular_tweets: True일 때 트위터 탭 중 Polular을 의미, False의 경우 Recent을 의미
  c.Store_csv = True #Store_json: 저장 시 JSON 포맷으로
  c.Hide_output = True #Hide_output: True일 때 주피터 노트북 콘솔에 output이 출력되는 옵션을 끔

  return c

In [ ]:
c = twint_find_new(word, startDate, endDate, outputFile)

twint.run.Search(c)

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 27.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 64.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 125.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 216.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 343.0 secs


In [ ]:
import pandas as pd
import json

df = pd.read_csv(outputFile)
df[["created_at", "id", "tweet", "retweets_count"]].head() # 편의상 4개의 칼럼만 출력

In [ ]:
#트윗들에 대해서만 tw_list에 저장

tw_list=[]
for i in range(len(df)):
  dt = df.loc[i,["tweet"]].str.strip()
  text = dt.tweet
  tw_list.append(dt.tweet)
  print(tw_list[i])